# Connect Four with Minimax AI

## Problem Statement

You are tasked with developing an AI for the classic **Connect Four** game, where two players alternate dropping pieces into a 6x7 grid. The goal is to get four consecutive pieces in a row, column, or diagonal. Your AI should use the **minimax algorithm** to determine the best moves **without implementing alpha-beta pruning**.

## Requirements

1. **Game Setup**: Implement a 6x7 grid for the Connect Four board. Each cell should either be empty or contain a piece from one of the two players (Player and AI).
   
2. **Piece Dropping**: Implement a function to drop a piece in the lowest available row within a chosen column. If the column is full, the move should be considered invalid.

3. **Winning Condition**: Develop a function to check if a player has won by getting four consecutive pieces in any row, column, or diagonal.

4. **Minimax Algorithm**:
   - Implement the **minimax algorithm** to evaluate all possible moves up to a given depth (e.g., 4).
   - The AI should aim to maximize its chances of winning while minimizing the player’s opportunities to win.
   - If a terminal state (win, loss, or draw) is reached at a particular depth, the algorithm should evaluate the board accordingly.
   
5. **Game Flow**:
   - The game should allow the player and AI to take alternate turns until one wins or the board is full (draw).
   - The player should be able to input a column to place their piece.
   - The AI should automatically choose a column based on the minimax evaluation.

6. **Display**: After each move, display the current board state to show piece placements.

## Objective

Design and implement the Connect Four game such that the AI uses the **minimax algorithm** to optimally decide its moves. The AI should play against the player and aim to win or force a draw by minimizing the player’s winning chances.


In [5]:
import math
import numpy as np
   
ROWS = 6
COLS = 7
PLAYER = 1  # User
AI = 2      # Computer
EMPTY = 0
WINDOW_LENGTH = 4

def create_board():
    return np.zeros((ROWS, COLS), dtype=int)

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[ROWS-1][col] == EMPTY

def get_next_open_row(board, col):
    for row in range(ROWS):
        if board[row][col] == EMPTY:
            return row

def print_board(board):
    print(np.flip(board, 0))

def winning_move(board, piece):
    # Check horizontal locations for win
    for c in range(COLS - 3):
        for r in range(ROWS):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True

    # Check vertical locations for win
    for c in range(COLS):
        for r in range(ROWS - 3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True

    # Check positively sloped diagonals
    for c in range(COLS - 3):
        for r in range(ROWS - 3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True

    # Check negatively sloped diagonals
    for c in range(COLS - 3):
        for r in range(3, ROWS):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True

    return False

#--------------------

def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER if piece == AI else AI

    if window.count(piece) == 4:
        score += 50
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 10
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 5


    return score
#---------------------

def score_position(board, piece):
    score = 0

    # Score center column higher as it is more promising.
    center_array = [int(i) for i in list(board[:, COLS//2])]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Score Horizontal
    for r in range(ROWS):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(COLS - 3):
            window = row_array[c:c+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Score Vertical
    for c in range(COLS):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(ROWS - 3):
            window = col_array[r:r+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Score positive sloped diagonal
    for r in range(ROWS - 3):
        for c in range(COLS - 3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    # Score negative sloped diagonal
    for r in range(3, ROWS):
        for c in range(COLS - 3):
            window = [board[r-i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    return score

def is_terminal_node(board):
    return winning_move(board, PLAYER) or winning_move(board, AI) or len(get_valid_locations(board)) == 0


def minimax(board, depth, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI):
                return (None, 100000000000000)
            elif winning_move(board, PLAYER):
                return (None, -100000000000000)
            else:  # Game is over, no more valid moves
                return (None, 0)
        else:  # Depth is zero
            return (None, score_position(board, AI))
    
    if maximizingPlayer:
        value = -math.inf
        best_col = np.random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI)
            new_score = minimax(b_copy, depth-1, False)[1]
            if new_score > value:
                value = new_score
                best_col = col
        return best_col, value
    else:  # Minimizing player
        value = math.inf
        best_col = np.random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER)
            new_score = minimax(b_copy, depth-1, True)[1]
            if new_score < value:
                value = new_score
                best_col = col
        return best_col, value

def get_valid_locations(board):
    valid_locations = []
    for col in range(COLS):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations


def best_move(board, depth=4):
    col, minimax_score = minimax(board, depth, True)
    return col

# Game setup
board = create_board()
game_over = False
turn = 0  # Player goes first

print_board(board)

## implement main function here
while not game_over:
    # Player move
    if turn == 0:
        valid_move = False
        while not valid_move:
            try:
                col = int(input("Player 1 Make your Selection (0-6): "))
                if col in get_valid_locations(board):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, PLAYER)
                    valid_move = True
                else:
                    print("Column is full or invalid. Try again.")
            except ValueError:
                print("Invalid input. Please enter an integer from 0 to 6.")
        if winning_move(board, PLAYER):
            print_board(board)
            print("PLAYER 1 WINS!!")
            game_over = True

    # AI move
    else:
        print("AI is thinking...")
        col = best_move(board, depth=4)
        if col is not None and is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, AI)
            if winning_move(board, AI):
                print_board(board)
                print("AI WINS!!")
                game_over = True
        else:
            print("No valid moves for AI. Game ends in a draw.")
            game_over = True

    print_board(board)
    print("---------------------------")
    # Check for draw
    if len(get_valid_locations(board)) == 0 and not game_over:
        print("Game is a draw!")
        game_over = True

    turn += 1
    turn = turn % 2


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]]
---------------------------
AI is thinking...
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 0 2 0 0 0]]
---------------------------


KeyboardInterrupt: Interrupted by user